In [ ]:
# default_exp core

# Simple Neo4j HTTP API Client

> Simple http client to connect neo4j server

In [ ]:
#hide
from nbdev.showdoc import *


In [ ]:
#export
import os,json,base64
import requests
import pandas as pd
from neo4jtools.utils import parse_result_graph

class SimpleNeo4jHTTPAPIClient:
    
    def __init__(self, url, db='neo4j', userid=None, passwd=None):
        self.url=url
        self.set_serverinfo(url)
        self.setdb(db)
        self.authtoken=None
        if userid is not None:
            self.authtoken=self.get_authtoken(userid, passwd)
        
    def set_serverinfo(self,url):
        resp=requests.get(url)
        obj=resp.json()
        self.bolt_routing=obj['bolt_routing']
        self.transaction=obj['transaction']
        self.bolt_direct=obj['bolt_direct']
        self.neo4j_version=obj['neo4j_version']
        self.neo4j_edition=obj['neo4j_edition']

    def setdb(self, db):
        self.db=db
        
    def execute_read_query(self, query, output_format=['row','graph']):
        url=self.transaction.format(databaseName=self.db) + '/commit'
        headers={
            "content-type": "application/json"
        }
        if self.authtoken is not None:
            headers['authorization']="Basic {}".format(self.authtoken)

        if isinstance(output_format, list):
            resultDataContents = output_format
        else:
            resultDataContents = [output_format]

        statement={
          "statements": [
            {
              "statement": query,
              "resultDataContents": resultDataContents
            }
          ]
        }
        resp=requests.post(url, 
                           data=json.dumps(statement), 
                           headers=headers)

        output= resp.json()

        if len(output['errors']) > 0: 
            raise Exception(output['errors'])
            
        output=output['results'][0]
        
        if output_format=='graph':
            nodes, edges = parse_result_graph(output['data'])
            output = {'nodes':nodes, 'edges':edges}
        
        return output
    
    def __repr__(self):
        return json.dumps({
            'classname':self.__class__.__name__,
            'url':self.url,
            'neo4j_version':self.neo4j_version,
            'neo4j_edition':self.neo4j_edition,
            'db':self.db,
            'auth': self.authtoken is not None
        })

    @staticmethod
    def get_authtoken(userid, passwd):
        authstr='{}:{}'.format(userid, passwd)
        b64token=base64.b64encode(authstr.encode())
        strtoken=b64token.decode()
        return strtoken      




## Simple example

In [ ]:
# Connect to a neo4j server without auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.3", "neo4j_edition": "community", "db": "neo4j", "auth": false}

In [ ]:
try:
    client.execute_read_query('match (n) return count(n);', output_format=['row'])
except Exception as e:
    assert e

In [ ]:
# Connect to a neo4j server with auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474', userid='neo4j',passwd='test')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.3", "neo4j_edition": "community", "db": "neo4j", "auth": true}

In [ ]:
result=client.execute_read_query('match (n) return count(n);', output_format='row')
result

{'columns': ['count(n)'], 'data': [{'row': [85852], 'meta': [None]}]}

# Example output_format='graph'

In [ ]:
result=client.execute_read_query('match p=()--() return p limit 3;', output_format='graph')

In [ ]:
result

{'nodes': [{'identifier': '4358',
   'license': 'CCO 1.0',
   'chromosome': '2',
   'name': 'MPV17',
   'description': 'mitochondrial inner membrane protein MPV17',
   'source': 'Entrez Gene:210321',
   'url': 'http://identifiers.org/ncbigene/4358',
   'label': 'Gene'},
  {'identifier': 'GO:0000002',
   'license': 'CC BY 4.0',
   'chromosome': nan,
   'name': 'mitochondrial genome maintenance',
   'description': nan,
   'source': 'Gene Ontology:2021-02-01',
   'url': 'http://purl.obolibrary.org/obo/GO:0000002',
   'label': 'BiologicalProcess'},
  {'identifier': '291',
   'license': 'CCO 1.0',
   'chromosome': '4',
   'name': 'SLC25A4',
   'description': 'solute carrier family 25 member 4',
   'source': 'Entrez Gene:210321',
   'url': 'http://identifiers.org/ncbigene/291',
   'label': 'Gene'},
  {'identifier': '55186',
   'license': 'CCO 1.0',
   'chromosome': '3',
   'name': 'SLC25A36',
   'description': 'solute carrier family 25 member 36',
   'source': 'Entrez Gene:210321',
   'url':

In [ ]:
pd.DataFrame(result['nodes'])

,identifier,license,chromosome,name,description,source,url,label
0,4358,CCO 1.0,2,MPV17,mitochondrial inner membrane protein MPV17,Entrez Gene:210321,http://identifiers.org/ncbigene/4358,Gene
1,GO:0000002,CC BY 4.0,NaN,mitochondrial genome maintenance,NaN,Gene Ontology:2021-02-01,http://purl.obolibrary.org/obo/GO:0000002,BiologicalProcess
2,291,CCO 1.0,4,SLC25A4,solute carrier family 25 member 4,Entrez Gene:210321,http://identifiers.org/ncbigene/291,Gene
3,55186,CCO 1.0,3,SLC25A36,solute carrier family 25 member 36,Entrez Gene:210321,http://identifiers.org/ncbigene/55186,Gene


In [ ]:
pd.DataFrame(result['edges'])

,type,start_identifier,start_name,end_identifier,end_name,license,unbiased,source,version
0,PARTICIPATES_GpBP,4358,MPV17,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,NCBI gene2go,2021-02-01
1,PARTICIPATES_GpBP,291,SLC25A4,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,NCBI gene2go,2021-02-01
2,PARTICIPATES_GpBP,55186,SLC25A36,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,NCBI gene2go,2021-02-01
